<a href="https://colab.research.google.com/github/secoxx/IE423/blob/main/task_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialize

In [3]:
import pandas as pd

## Load Data

Remember the retail transactions data set that we used for customer segmentation...

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/black_friday/train.csv')

In [6]:
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


## Prepare Data

In [7]:
X = df.groupby('User_ID')['Product_ID'].apply(list)
X

User_ID
1000001    [P00069042, P00248942, P00087842, P00085442, P...
1000002    [P00285442, P00112842, P00293242, P00289342, P...
1000003    [P00193542, P00132842, P0098342, P00010242, P0...
1000004    [P00184942, P00346142, P0097242, P00046742, P0...
1000005    [P00274942, P00251242, P00014542, P00031342, P...
                                 ...                        
1006036    [P00237642, P00250342, P00173242, P00255342, P...
1006037    [P00067342, P00111842, P00323642, P00177442, P...
1006038    [P00034742, P00251242, P00086042, P00109542, P...
1006039    [P00114042, P00199442, P00088542, P00254242, P...
1006040    [P00184042, P00158742, P00001742, P00148642, P...
Name: Product_ID, Length: 5891, dtype: object

## Build Model

### Association Rule Mining

Suppose we are interested in the relationship A -> B.  Then:

* **Support**: Fraction of transactions that contain both A and B = Freq(A,B)

* **Confidence**: How often B appears given A appeared = Support / Freq(A)

* **Lift**: Confidence given B appeared = Confidence / Freq(B)

In [8]:
# # Encode data as transaction matrix
# from mlxtend.preprocessing import TransactionEncoder

# mdlRetTe = TransactionEncoder()
# mdlRetTe_array = mdlRetTe.fit(dfRetList).transform(dfRetList)
# dfRtb = pd.DataFrame(mdlRetTe_array, columns=mdlRetTe.columns_)
# dfRtb
# Encode data as transaction matrix
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd

# Convert all items in transactions to strings
#dfRetList_str = dfRetList.apply(lambda transaction: [str(item) for item in transaction])

mdlTe = TransactionEncoder()
mdlTe_array = mdlTe.fit(X).transform(X) # Use the string-converted list
dfTm = pd.DataFrame(mdlTe_array, columns=mdlTe.columns_)
dfTm

,P00000142,P00000242,P00000342,P00000442,P00000542,P00000642,P00000742,P00000842,P00000942,P00001042,...,P0098942,P0099042,P0099142,P0099242,P0099342,P0099442,P0099642,P0099742,P0099842,P0099942
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5886,False,False,False,True,False,True,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
5887,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5888,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5889,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
%%time
# Determine the items and itemsets with at least 1% support (generates all itemsets and then filters by support)

from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(dfTm, min_support=0.01, use_colnames=True)
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
!pip3 install mlxtend --upgrade

In [10]:
%%time
# Speed up by using pattern fragment growth method for mining frequent itemsets (uses density to find good itemsets)

from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets = fpgrowth(dfTm, min_support=0.01, use_colnames=True)
frequent_itemsets

CPU times: user 22min 48s, sys: 7.17 s, total: 22min 55s
Wall time: 23min 28s


,support,itemsets
0,0.274147,(P00025442)
1,0.244441,(P00184942)
2,0.238669,(P00059442)
3,0.234765,(P00255842)
4,0.231540,(P00117942)
...,...,...
1351677,0.010525,"(P00190942, P00112142)"
1351678,0.010015,"(P00190942, P00190142)"
1351679,0.010355,"(P00190942, P00035542)"
1351680,0.010355,"(P00190942, P00046742)"


The cell above takes 23 minutes.

In [11]:
# Evaluate the metrics, and filter the items and itemsets that have at least 70% confidence

from mlxtend.frequent_patterns import association_rules

a_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
a_rules.sort_values(by=['confidence'],ascending=False,inplace=True)
a_rules.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
57,"(P00144642, P00329542, P00128942)",(P00057642),0.013071,0.249533,0.010185,0.779221,3.122714,0.006923,3.399173,0.688768
182,"(P00147942, P00112142, P00021742)",(P00110742),0.013410,0.273638,0.010355,0.772152,2.821803,0.006685,3.187923,0.654392
34,"(P00117942, P00070042, P00277642)",(P00145042),0.015617,0.238669,0.012052,0.771739,3.233510,0.008325,3.335354,0.701697
134,"(P00127742, P00058042, P00102642)",(P00057642),0.013750,0.249533,0.010525,0.765432,3.067456,0.007094,3.199359,0.683393
186,"(P00112142, P00057942, P00021742)",(P00110742),0.013580,0.273638,0.010355,0.762500,2.786531,0.006639,3.058367,0.649957


From the first row we see that when someone purchases P00144642, P00329542, P00128942 they also purchase P00057642. When we look at the first row's metrics, we see that although support is low (1%), confidence is high (77%); meaning that even though these two items did not appear together a lot, when antecedent was present mostly consequent was also present. Lift seems also low, meaning that consequent showing up without the antecedent is not very likely.

## Conclusion

* By association rule mining, we managed to find out itemsets that are requently bought together.
* We looked at metrics such as support, confidence and lift. We saw that the most related itemsets had high confidence levels even if they had low support. Meaning that even though the itemsets were not observed that often; when they were ıbserved, they were observed together. When we looked at lift, we saw that them being observed alone were not very likely.